In [1]:
from pyspark.sql import SparkSession 

spark = SparkSession.Builder().appName('MapFunctions').getOrCreate()

24/06/16 12:19:31 WARN Utils: Your hostname, codebase resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/06/16 12:19:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/16 12:19:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/16 12:19:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
from pyspark.sql.types import StructType, StringType,StructField , MapType, IntegerType
from pyspark.sql.functions import col, lit, explode, map_keys, map_values

data = [
    ('rohit', {'Skill' : 'SQL', 'Rating' : 5, 'ready' : True}),
    ('ajay', {'Skill' : 'Python', 'Rating' : 5}),
    ('dhananjay', {'Skill' : 'Azure', 'Rating' : 5}),
]

# schema = StructType(
#     [
#         StructField('name', StringType()),
#         StructField('Skill-Rating', MapType(StringType(), IntegerType()))
#     ]
# )

# Or

# If we don't enforce the schema as Map Type it will automatically take it as MapType() as the data is given in
# python dictionary (Dict) 

schema = ['name', 'skills-rating']

df = spark.createDataFrame(data, schema)
df.show(truncate= False)
df.printSchema()

+---------+------------------------------------------+
|name     |skills-rating                             |
+---------+------------------------------------------+
|rohit    |{Skill -> SQL, ready -> true, Rating -> 5}|
|ajay     |{Skill -> Python, Rating -> 5}            |
|dhananjay|{Skill -> Azure, Rating -> 5}             |
+---------+------------------------------------------+

root
 |-- name: string (nullable = true)
 |-- skills-rating: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



##### explode() functions --> to get the key and values in different columns

In [12]:
df.select("name", explode(df['skills-rating'])).distinct().show()

+---------+------+------+
|     name|   key| value|
+---------+------+------+
|    rohit| ready|  true|
|    rohit| Skill|   SQL|
|    rohit|Rating|     5|
|     ajay|Rating|     5|
|     ajay| Skill|Python|
|dhananjay| Skill| Azure|
|dhananjay|Rating|     5|
+---------+------+------+



In [13]:
df.withColumn('keys', map_keys(df['skills-rating'])).show(truncate=False)

+---------+------------------------------------------+----------------------+
|name     |skills-rating                             |keys                  |
+---------+------------------------------------------+----------------------+
|rohit    |{Skill -> SQL, ready -> true, Rating -> 5}|[Skill, ready, Rating]|
|ajay     |{Skill -> Python, Rating -> 5}            |[Skill, Rating]       |
|dhananjay|{Skill -> Azure, Rating -> 5}             |[Skill, Rating]       |
+---------+------------------------------------------+----------------------+



In [15]:
df.withColumn('values', map_values(df['skills-rating'])).show(truncate=False)

+---------+------------------------------------------+--------------+
|name     |skills-rating                             |values        |
+---------+------------------------------------------+--------------+
|rohit    |{Skill -> SQL, ready -> true, Rating -> 5}|[SQL, true, 5]|
|ajay     |{Skill -> Python, Rating -> 5}            |[Python, 5]   |
|dhananjay|{Skill -> Azure, Rating -> 5}             |[Azure, 5]    |
+---------+------------------------------------------+--------------+



In [16]:
spark.stop()